<img align="left" style="padding-right:10px; width:150px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/mindboggle_logo_200px_600dpi.jpg">
This jupyter notebook provides a tutorial for the Mindboggle brain image analysis software.<br>&nbsp;&nbsp;&nbsp;&nbsp;Website: http://mindboggle.info<br>&nbsp;&nbsp;&nbsp;&nbsp;Reference: http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005350#sec007

# Mindboggle tutorial

1. [Mindboggle processing steps](#processing)
2. [Run "mindboggle --help" on the command line](#help)
3. [Mindboggle on the command line](#command)
4. [Mindboggle Python library](#library)
5. [Run individual functions](#functions)

<a id="processing"></a>

# Mindboggle processing steps

<img style="padding-right:10px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/mindboggle_nipype_flow.png">

## Output
Mindboggle takes in (FreeSurfer- and optionally ANTs-) preprocessed T1-weighted MRI data, and outputs nifti volumes, vtk surfaces, and csv tables containing label, feature, and shape information for further analysis:

- **labels/**:  *integer-labeled vtk surfaces and nifti volumes*
- **features/**:  *integer-labeled sulci or fundi on vtk surfaces*
- **shapes/**:  *float shape value for every point on vtk surfaces*
- **tables/**:  *csv tables of shape values for every label/feature/vertex*

<br><br>

## Processing steps

### 1. Combine FreeSurfer and ANTs gray/white segmentations:
<br>
<img style="padding-right:10px; width:600px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/papers/Figure2_FSvsANTs_segmentation_white_188mm_600dpi.png">
<br>

### 2. Fill hybrid segmentation with (FreeSurfer- or ANTs-registered) labels.
### 3. Compute volume shape measures for each labeled region:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>volume</b>
<br>
<img style="padding-right:10px; width:600px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/posters/roygbiv-volume.png">
<br>

### 4. Compute surface shape measures for every cortical mesh vertex
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>surface area</b>
<br>
<img style="padding-right:10px; width:600px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/papers/Figure4_area_182mm_300dpi.png">
<br><br>
<img style="padding-right:10px; width:600px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/papers/HBM2015_ROYGBIV.png">
<br>

<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>geodesic depth and travel depth</b>
<br>
<img style="padding-right:10px; width:600px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/papers/Figure6_geodesic_and_travel_depth_on_sulci_190mm_600dpi.png">
<br>

<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>mean curvature</b>
<br>
<img style="padding-right:10px; width:600px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/papers/Mindboggle_bioRxiv2016_Fig5.png">
<br>

### 5. Extract cortical surface features:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>folds</b> (left and upper right, with manual labels) 
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>sulci</b> (lower right)
<br>
<img style="padding-right:10px; width:600px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/papers/Mindboggle_bioRxiv2016_Fig7.png">

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>fundi</b>
<img style="padding-right:10px; width:300px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/papers/Figure8_fundus_views_78mm_600dpi_rotated.png">
<br>

### 6. For each cortical surface label/sulcus, compute:
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>Zernike moments</b>
<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<b>Laplace-Beltrami spectrum</b> (2nd, 3rd, and 9th spectral components shown for two brains):
<br>
<br>
<img style="padding-right:10px; width:600px;" src="https://raw.githubusercontent.com/nipy/mindboggle-assets/master/images/papers/Figure9_LaplaceBeltrami3rd6th9thSpectralComponents_190mm_600dpi.png">

### 7. Compute statistics for each shape measure across vertices for each label/feature:
    - median
    - median absolute deviation
    - mean
    - standard deviation
    - skew
    - kurtosis
    - lower and upper quartiles
<br>

<a id="help"></a>

# Run "mindboggle --help" on the command line

First, let's see what command-line options Mindboggle provides:

In [1]:
! mindboggle --help

/bin/sh: mindboggle: command not found


<a id="command"></a>

# Mindboggle on the command line

The following command computes shape measures for cortical surface labels and sulci from [example FreeSurfer and ANTs data](https://osf.io/3xfb8/?action=download&version=1). Without adding more restrictive arguments, this command takes an hour or two to run.

mindboggle /home/jovyan/work/mindboggle_input_example/freesurfer/subjects/arno<br>&nbsp;&nbsp;&nbsp;&nbsp; --ants /home/jovyan/work/mindboggle_input_example/ants/subjects/arno/antsBrainSegmentation.nii.gz<br>&nbsp;&nbsp;&nbsp;&nbsp; --out /home/jovyan/work/mindboggled<br>&nbsp;&nbsp;&nbsp;&nbsp; --roygbiv<br><br><br>

<a id="library"></a>

# Mindboggle Python library

Rather than call Mindboggle from the command line, we can also call individual Python functions within the Mindboggle library, which includes the following files in mindboggle/mindboggle/:

- **mio**/  *-- input/output functions*
    - **[colors](https://github.com/nipy/mindboggle/blob/master/mindboggle/mio/colors.py)**.py  *-- colormap-related functions*
    - **[convert_volumes](https://github.com/nipy/mindboggle/blob/master/mindboggle/mio/convert_volumes.py)**.py  *-- read/write nifti volume files*
    - **[fetch_data](https://github.com/nipy/mindboggle/blob/master/mindboggle/mio/fetch_data.py)**.py  *-- fetch data from a URL or from third party software*
    - **[labels](https://github.com/nipy/mindboggle/blob/master/mindboggle/mio/labels.py)**.py  *-- information about labeling protocols*
    - **[plots](https://github.com/nipy/mindboggle/blob/master/mindboggle/mio/plots.py)**.py  *-- plot functions*
    - **[tables](https://github.com/nipy/mindboggle/blob/master/mindboggle/mio/tables.py)**.py  *-- read/write tables*
    - **[vtks](https://github.com/nipy/mindboggle/blob/master/mindboggle/mio/vtks.py)**.py  *-- read/write VTK surface files*

- **guts**/  *-- the "guts" underlying feature extraction and labeling code*
    - **[compute](https://github.com/nipy/mindboggle/blob/master/mindboggle/guts/compute.py)**.py  *-- compute distances, etc.*
    - **[graph](https://github.com/nipy/mindboggle/blob/master/mindboggle/guts/graph.py)**.py  *-- graph operations*
    - **[kernels](https://github.com/nipy/mindboggle/blob/master/mindboggle/guts/kernels.py)**.py  *-- kernels for graph operations*
    - **[mesh](https://github.com/nipy/mindboggle/blob/master/mindboggle/guts/mesh.py)**.py  *-- operate on surface mesh vertices*
    - **[paths](https://github.com/nipy/mindboggle/blob/master/mindboggle/guts/paths.py)**.py  *-- connect surface mesh vertices*
    - **[rebound](https://github.com/nipy/mindboggle/blob/master/mindboggle/guts/rebound.py)**.py  *-- adjust label borders on a surface mesh*
    - **[relabel](https://github.com/nipy/mindboggle/blob/master/mindboggle/guts/relabel.py)**.py  *-- relabel surface or volume files*
    - **[segment](https://github.com/nipy/mindboggle/blob/master/mindboggle/guts/segment.py)**.py  *-- segment a surface mesh*

- **shapes**/  *-- shape measurement functions
    - **[surface_shapes](https://github.com/nipy/mindboggle/blob/master/mindboggle/shapes/surface_shapes.py)**.py  *-- compute surface shapes (calls C++ library below)*
    - **[volume_shapes](https://github.com/nipy/mindboggle/blob/master/mindboggle/shapes/volume_shapes.py)**.py  *-- compute volumes and thicknesses*
    - **[laplace_beltrami](https://github.com/nipy/mindboggle/blob/master/mindboggle/shapes/laplace_beltrami.py)**.py  *-- compute a Laplace-Beltrami spectrum*
    - **[zernike/zernike](https://github.com/nipy/mindboggle/blob/master/mindboggle/shapes/zernike/zernike.py)**.py  *-- compute Zernike moments of a collection of vertices*
    - **[likelihood](https://github.com/nipy/mindboggle/blob/master/mindboggle/shapes/likelihood.py)**.py  *-- compute (fundus) likelihood values*

- **features**/  *-- higher-level feature extraction (folds, fundi, sulci)*
    - **[folds](https://github.com/nipy/mindboggle/blob/master/mindboggle/features/folds.py)**.py  *-- extract surface folds*
    - **[fundi](https://github.com/nipy/mindboggle/blob/master/mindboggle/features/fundi.py)**.py  *-- extract fundus curves from folds*
    - **[sulci](https://github.com/nipy/mindboggle/blob/master/mindboggle/features/sulci.py)**.py  *-- extract sulci from folds*

<!--
    - **thirdparty/**  *-- third-party code*
    - **[ants](https://github.com/nipy/mindboggle/blob/master/mindboggle/thirdparty/ants.py)**.py  *-- call ANTs commands*
    - **[vtkviewer](https://github.com/nipy/mindboggle/blob/master/mindboggle/thirdparty/vtkviewer.py)**.py  *-- VTK viewer (by Hal Canary)*
  - mindboggle/vtk_cpp_tools  *-- C++ tools for measuring shapes on VTK surface files*
    - **[area](https://github.com/nipy/mindboggle/blob/master/vtk_cpp_tools/area/PointAreaMain.cpp)**/
    - **[curvature](https://github.com/nipy/mindboggle/blob/master/vtk_cpp_tools/curvature/CurvatureMain.cpp)**/
    - **[geodesic_depth](https://github.com/nipy/mindboggle/blob/master/vtk_cpp_tools/geodesic_depth/GeodesicDepthMain.cpp)**/
    - **[travel_depth](https://github.com/nipy/mindboggle/tree/master/vtk_cpp_tools/travel_depth)**/
    - **[gradient](https://github.com/nipy/mindboggle/blob/master/vtk_cpp_tools/gradient/GradientMain.cpp)**/
    - **[surface_overlap](https://github.com/nipy/mindboggle/blob/master/vtk_cpp_tools/surface_overlap/SurfaceOverlapMain.cpp)**/
-->
<br>
<br>

<a id="functions"></a>

# Run individual functions

Let's run some functions within Mindboggle's Python library.  The following code snippets are adapted from the above files' docstrings.

## Example: Compute statistics of depth measures in sulcus folds
### Measure travel depth for every vertex of a brain's left hemisphere
Convert a FreeSurfer surface file to VTK format:

In [ ]:
from mindboggle.mio.vtks import freesurfer_surface_to_vtk
subject_path = '/home/jovyan/work/mindboggle_input_example/freesurfer/subjects/arno/'
surface_file = freesurfer_surface_to_vtk(surface_file=subject_path + 'surf/lh.pial', 
                                         orig_file=subject_path + 'mri/orig.mgz',
                                         output_vtk='lh.pial.vtk')

Compute travel_depth for every vertex of the mesh in the VTK file:

In [ ]:
import os
from mindboggle.shapes.surface_shapes import travel_depth
from mindboggle.mio.vtks import read_scalars
ccode_path = '/opt/vtk_cpp_tools'
command = os.path.join(ccode_path, 'travel_depth', 'TravelDepthMain')
depth_file = travel_depth(command=command,
                          surface_file=surface_file,
                          verbose=True)
depths, name = read_scalars(depth_file)

Plot the depth values in 3-D:  <b>UNDER CONSTRUCTION<b>

In [ ]:
import numpy as np
import nilearn.plotting
depths_array = np.asarray(depths)
fs_surf_mesh = subject_path + 'surf/lh.pial'
nilearn.plotting.plot_surf(surf_mesh=fs_surf_mesh, surf_map=depths_array,
                           bg_map=None, hemi='left', view='lateral',
                           cmap=None, colorbar=True,
                           avg_method='mean', threshold=None,
                           alpha='auto', bg_on_data=False, darkness=1,
                           vmin=None, vmax=None, cbar_vmin=None, cbar_vmax=None,
                           title=None, output_file=None, axes=None, figure=None)

### Extract folds based on depth and curvature
Plot a histogram of the depth values:

In [ ]:
%matplotlib inline
from mindboggle.mio.plots import histograms_of_lists
histograms_of_lists(columns=[depths],
                    column_name='Depth values',
                    ignore_columns=[],
                    nbins=100,
                    axis_limits=[],
                    titles='depth values')

Find a depth threshold to extract folds from the surface:

In [ ]:
from mindboggle.features.folds import find_depth_threshold
depth_threshold, bins, bin_edges = find_depth_threshold(depth_file=depth_file,
                                                        min_vertices=10000,
                                                        verbose=True)
depth_threshold

Extract folds with the depth threshold:

In [ ]:
from mindboggle.features.folds import extract_folds
folds, n_folds, folds_file = extract_folds(depth_file=depth_file,
                                           depth_threshold=depth_threshold,
                                           min_fold_size=50,
                                           save_file=True,
                                           output_file='folds.vtk',
                                           background_value=-1,
                                           verbose=True)

Remove all vertices but the folds:

In [ ]:
from mindboggle.mio.vtks import rewrite_scalars
rewrite_scalars(input_vtk=folds_file,
                output_vtk='rewrite_scalars.vtk',
                new_scalars=[folds],
                new_scalar_names=['folds'],
                filter_scalars=folds,
                background_value=-1)

Plot the folds in 3-D:  <b>UNDER CONSTRUCTION<b>

In [ ]:
folds_array = np.asarray(folds)
nilearn.plotting.plot_surf(surf_mesh=fs_surf_mesh, surf_map=folds_array,
                           bg_map=None, hemi='left', view='lateral',
                           cmap=None, colorbar=True,
                           avg_method='mean', threshold=None,
                           alpha='auto', bg_on_data=False, darkness=1,
                           vmin=None, vmax=None, cbar_vmin=None, cbar_vmax=None,
                           title=None, output_file=None, axes=None, figure=None)

### Extract sulci from folds

Load a FreeSurfer .annot file and save as a VTK format file:

In [ ]:
from mindboggle.mio.vtks import freesurfer_annot_to_vtk
labels, label_file = freesurfer_annot_to_vtk(annot_file=subject_path + 'label/lh.aparc.annot',
                                             vtk_file=surface_file,
                                             output_vtk='lh.aparc.annot.vtk',
                                             background_value=-1)

Relabel surface labels to match expected volume labels:

In [ ]:
from mindboggle.guts.relabel import relabel_surface
from mindboggle.mio.labels import DKTprotocol
dkt = DKTprotocol()
relabel_file = relabel_surface(vtk_file=label_file,
                               hemi='lh', 
                               old_labels=dkt.DKT31_numbers, 
                               new_labels=[],
                               erase_remaining=True, 
                               erase_labels=[0], 
                               erase_value=-1, 
                               output_file='relabeled.vtk')

Extract sulci from folds using pairs of labels in the DKT labeling protocol (SLOW):

In [ ]:
from mindboggle.features.sulci import extract_sulci
sulci, n_sulci, sulci_file = extract_sulci(labels_file=relabel_file,
                                           folds_or_file=folds,
                                           hemi='lh',
                                           min_boundary=10,
                                           sulcus_names=[],
                                           save_file=True,
                                           output_file='sulci.vtk',
                                           background_value=-1,
                                           verbose=True)
n_sulci

Plot the sulci in 3-D:  <b>UNDER CONSTRUCTION<b>

In [ ]:
sulci_array = np.asarray(sulci)
nilearn.plotting.plot_surf(surf_mesh=fs_surf_mesh, surf_map=sulci_array,
                           bg_map=None, hemi='left', view='lateral',
                           cmap=None, colorbar=True,
                           avg_method='mean', threshold=None,
                           alpha='auto', bg_on_data=False, darkness=1,
                           vmin=None, vmax=None, cbar_vmin=None, cbar_vmax=None,
                           title=None, output_file=None, axes=None, figure=None)

### Compute statistics on sulcus depth values

In [ ]:
from mindboggle.mio.tables import write_shape_stats
label_table, sulcus_table, fundus_table = write_shape_stats(labels_or_file=[], 
    sulci=sulci, fundi=[], affine_transform_files=[], inverse_booleans=[], 
    transform_format='itk', area_file='', normalize_by_area=False, 
    mean_curvature_file='',
    travel_depth_file=depth_file, geodesic_depth_file='',
    freesurfer_thickness_file='', freesurfer_curvature_file='',
    freesurfer_sulc_file='',
    labels_spectra=[], labels_spectra_IDs=[],
    sulci_spectra=[], sulci_spectra_IDs=[],
    labels_zernike=[], labels_zernike_IDs=[],
    sulci_zernike=[], sulci_zernike_IDs=[],
    exclude_labels=[-1], verbose=True)

Show statistical summary table of sulcus depth values:

In [ ]:
import pandas as pd
pd.read_csv(sulcus_table)